In [ ]:
# importing pandas

import pandas as pd

In [2]:
# importing data

df = pd.read_excel('./data.xlsx')

In [3]:
df.head()

,Region,Period,ResourcesNeeded,Resources
0,East,1,1,"19282,19355,23860,27605,32001,50762,63756,6778..."
1,East,2,4,"19282,19355,23860,27605,32001,50762,63756,6778..."
2,East,3,1,"19282,19355,23860,27605,32001,50762,63756,6778..."
3,East,4,6,"19282,19355,23860,27605,32001,50762,63756,6778..."
4,East,5,5,"19282,19355,23860,27605,32001,50762,63756,6778..."


In [4]:
# grouping by region and resources

resources_region = df.groupby('Region').agg({'Resources' : min})

C:\Users\CharlesYi\AppData\Local\Temp\ipykernel_28616\3185566678.py:3: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  resources_region = df.groupby('Region').agg({'Resources' : min})


In [5]:
# splitting resources into rows

resources_region['Resources'] = resources_region['Resources'].str.split(',')

In [6]:
# exploding list into rows

resources_region = resources_region.explode('Resources')

In [7]:
# resetting index

resources_region = resources_region.reset_index()

resources_region.tail()

,Region,Resources
49,West,79074
50,West,85029
51,West,87972
52,West,89128
53,West,94417


In [8]:
# adding record to table per region

resources_region['record'] = resources_region.groupby('Region').cumcount() + 1

resources_region

,Region,Resources,record
0,East,19282,1
1,East,19355,2
2,East,23860,3
3,East,27605,4
4,East,32001,5
5,East,50762,6
6,East,63756,7
7,East,67786,8
8,East,70849,9
9,East,74680,10


In [41]:
# pivoting to original df, scaffolding based on resources needed

# creating function to scaffold each row

rows = []

def scaffold (row):
    record = 1
    if row['ResourcesNeeded'] == 0:
        new_row_1 = {
            'Region' : row['Region'],
            'Period' : row['Period'],
            'ResourcesNeeded' : row['ResourcesNeeded'],
            'Resources' : row['Resources'],
                   }
        rows.append(new_row_1)
    else:
        for i in range(row['ResourcesNeeded']):
            new_row_2 = {
                'Region' : row['Region'],
                'Period' : row['Period'],
                'ResourcesNeeded' : row['ResourcesNeeded'],
                'Resources' : row['Resources'],
                   }
            rows.append(new_row_2)

In [38]:
# applying function by iterating through each row

for index, row in df.iterrows():
    scaffold(row)

In [33]:
# converting rows to df

df_transformed = pd.json_normalize(rows)

df_transformed

,Region,Period,ResourcesNeeded,Resources
0,East,1,1,"19282,19355,23860,27605,32001,50762,63756,6778..."
1,East,2,4,"19282,19355,23860,27605,32001,50762,63756,6778..."
2,East,2,4,"19282,19355,23860,27605,32001,50762,63756,6778..."
3,East,2,4,"19282,19355,23860,27605,32001,50762,63756,6778..."
4,East,2,4,"19282,19355,23860,27605,32001,50762,63756,6778..."
...,...,...,...,...
309,West,12,5,"13035,31242,38361,42953,47248,58366,76212,7907..."
310,West,12,5,"13035,31242,38361,42953,47248,58366,76212,7907..."
311,West,12,5,"13035,31242,38361,42953,47248,58366,76212,7907..."
312,West,12,5,"13035,31242,38361,42953,47248,58366,76212,7907..."


In [18]:
# adding record id per region

def record_generator(region):
    record_list = []
    record_id = 1
    
    for index, row in region.iterrows():
        
        if row['ResourcesNeeded'] == 0:
            record_list.append(0)
        else:
            record_list.append(record_id)
        
            record_id += 1
            
    return pd.Series(record_list, index = region.index)

df_transformed['record'] = df_transformed.groupby('Region').apply(record_generator).reset_index(level=0, drop=True)

df_transformed

,Region,Period,ResourcesNeeded,Resources,record
0,East,1,1,"19282,19355,23860,27605,32001,50762,63756,6778...",1
1,East,2,4,"19282,19355,23860,27605,32001,50762,63756,6778...",2
2,East,2,4,"19282,19355,23860,27605,32001,50762,63756,6778...",3
3,East,2,4,"19282,19355,23860,27605,32001,50762,63756,6778...",4
4,East,2,4,"19282,19355,23860,27605,32001,50762,63756,6778...",5
...,...,...,...,...,...
309,West,12,5,"13035,31242,38361,42953,47248,58366,76212,7907...",64
310,West,12,5,"13035,31242,38361,42953,47248,58366,76212,7907...",65
311,West,12,5,"13035,31242,38361,42953,47248,58366,76212,7907...",66
312,West,12,5,"13035,31242,38361,42953,47248,58366,76212,7907...",67


In [19]:
resources_region.tail()

,Region,Resources,record
49,West,79074,8
50,West,85029,9
51,West,87972,10
52,West,89128,11
53,West,94417,12


In [20]:
df_transformed.tail()

,Region,Period,ResourcesNeeded,Resources,record
309,West,12,5,"13035,31242,38361,42953,47248,58366,76212,7907...",64
310,West,12,5,"13035,31242,38361,42953,47248,58366,76212,7907...",65
311,West,12,5,"13035,31242,38361,42953,47248,58366,76212,7907...",66
312,West,12,5,"13035,31242,38361,42953,47248,58366,76212,7907...",67
313,West,12,5,"13035,31242,38361,42953,47248,58366,76212,7907...",68


In [21]:
# joining resources_region and df_transformed on region and record

df_joined = resources_region.merge(df_transformed, left_on = ['record', 'Region'], right_on = ['record', 'Region'], how = 'right')

In [22]:
df_joined

,Region,Resources_x,record,Period,ResourcesNeeded,Resources_y
0,East,19282,1,1,1,"19282,19355,23860,27605,32001,50762,63756,6778..."
1,East,19355,2,2,4,"19282,19355,23860,27605,32001,50762,63756,6778..."
2,East,23860,3,2,4,"19282,19355,23860,27605,32001,50762,63756,6778..."
3,East,27605,4,2,4,"19282,19355,23860,27605,32001,50762,63756,6778..."
4,East,32001,5,2,4,"19282,19355,23860,27605,32001,50762,63756,6778..."
...,...,...,...,...,...,...
309,West,NaN,64,12,5,"13035,31242,38361,42953,47248,58366,76212,7907..."
310,West,NaN,65,12,5,"13035,31242,38361,42953,47248,58366,76212,7907..."
311,West,NaN,66,12,5,"13035,31242,38361,42953,47248,58366,76212,7907..."
312,West,NaN,67,12,5,"13035,31242,38361,42953,47248,58366,76212,7907..."


In [23]:
# selecting relevant columns

df_joined = df_joined[['Region', 'Resources_x', 'Period', 'ResourcesNeeded']]

df_joined

,Region,Resources_x,Period,ResourcesNeeded
0,East,19282,1,1
1,East,19355,2,4
2,East,23860,2,4
3,East,27605,2,4
4,East,32001,2,4
...,...,...,...,...
309,West,NaN,12,5
310,West,NaN,12,5
311,West,NaN,12,5
312,West,NaN,12,5


In [24]:
# changing resources to string

df_joined['Resources_x'] = df_joined['Resources_x'].astype('str')

# grouping resources by region

# creating custom function to concat non-null resources

def concat(resources):
    return ','.join([resource for resource in resources if resource != 'nan' and resource != ''])

df_final = df_joined.groupby(['Region', 'Period', 'ResourcesNeeded'])['Resources_x'].agg(concat).reset_index()

# renaming resources column

df_final = df_final.rename(columns = {'Resources_x' : 'Resources'})

df_final

C:\Users\CharlesYi\AppData\Local\Temp\ipykernel_28616\3601611522.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_joined['Resources_x'] = df_joined['Resources_x'].astype('str')


,Region,Period,ResourcesNeeded,Resources
0,East,1,1,19282
1,East,2,4,"19355,23860,27605,32001"
2,East,3,1,50762
3,East,4,6,"63756,67786,70849,74680,75064,85398"
4,East,5,5,"89675,90271,96677"
5,East,6,2,
6,East,7,4,
7,East,8,2,
8,East,9,0,
9,East,10,4,


In [25]:
# finding if resources were met

result = []

for index, row in df_final.iterrows():
    resources_list = [res for res in row['Resources'].split(',') if res != '']
    if len(resources_list) == row['ResourcesNeeded']:
                                result.append('Yes')
    else:
                                result.append('No')
                                  

df_final['Resources Need Met'] = result

df_final

,Region,Period,ResourcesNeeded,Resources,Resources Need Met
0,East,1,1,19282,Yes
1,East,2,4,"19355,23860,27605,32001",Yes
2,East,3,1,50762,Yes
3,East,4,6,"63756,67786,70849,74680,75064,85398",Yes
4,East,5,5,"89675,90271,96677",No
5,East,6,2,,No
6,East,7,4,,No
7,East,8,2,,No
8,East,9,0,,Yes
9,East,10,4,,No
